This notebook is adapted from the _movie-graph_ playbook, which is part of the Neo4j Community Edition and which can be started there via `:play movie-graph`.

# Movie Graph

#### Pop-cultural connections between actors and movies


The Movie Graph is a mini graph application containing actors and directors that are related through the movies they've collaborated on.

This guide will show you how to:

  1. Create: insert movie data into the graph
  2. Find: retrieve individual movies and actors
  3. Query: discover related actors and directors
  4. Solve: the Bacon Path

## Create

In the file `create_movie_graph.cql` contains a giant code block containing a single Cypher query statement composed of multiple CREATE clauses. This will create the movie graph.

  1. Run (`CTRL+Return`) the Bash script below to create the data in your DB.
  2. Wait for the query to finish
  3. **WARNING**: This adds data to the current database, each time it is run!

In [21]:
%%bash
cat create_movie_graph.cql | cypher-shell -u neo4j -p class --format plain

## Find

Example queries for finding individual nodes.

  1. Execute each of the following queries (CTRL+Return)
  2. Notice the syntax pattern
  3. Try looking for other movies or actors

In [1]:
MATCH (tom {name: "Tom Hanks"})
RETURN tom

+-------------------------------------------------------+
| tom                                                   |
+-------------------------------------------------------+
| (:Person {name: "Tom Hanks", _id_: 1272, born: 1956}) |
+-------------------------------------------------------+

1 row available after 1 ms, consumed after another 1 ms

In [2]:
MATCH (cloudAtlas {title: "Cloud Atlas"})
RETURN cloudAtlas

+-------------------------------------------------------------------------------------------------+
| cloudAtlas                                                                                      |
+-------------------------------------------------------------------------------------------------+
| (:Movie {tagline: "Everything is connected", _id_: 1306, title: "Cloud Atlas", released: 2012}) |
+-------------------------------------------------------------------------------------------------+

1 row available after 1 ms, consumed after another 0 ms

In [3]:
MATCH (people:Person)
RETURN people.name LIMIT 10

+----------------------+
| people.name          |
+----------------------+
| "Keanu Reeves"       |
| "Carrie-Anne Moss"   |
| "Laurence Fishburne" |
| "Hugo Weaving"       |
| "Lilly Wachowski"    |
| "Lana Wachowski"     |
| "Joel Silver"        |
| "Emil Eifrem"        |
| "Charlize Theron"    |
| "Al Pacino"          |
+----------------------+

10 rows available after 10 ms, consumed after another 1 ms

In [4]:
MATCH (nineties:Movie) 
WHERE nineties.released >= 1990 AND nineties.released < 2000 
RETURN nineties.title LIMIT(10)

+--------------------------+
| nineties.title           |
+--------------------------+
| "The Matrix"             |
| "The Devil's Advocate"   |
| "A Few Good Men"         |
| "As Good as It Gets"     |
| "What Dreams May Come"   |
| "Snow Falling on Cedars" |
| "You've Got Mail"        |
| "Sleepless in Seattle"   |
| "Joe Versus the Volcano" |
| "When Harry Met Sally"   |
+--------------------------+

10 rows available after 98 ms, consumed after another 0 ms

## Query

Finding patterns within the graph.

  1. Actors are people who acted in movies
  2. Directors are people who directed a movie
  3. What other relationships exist?


In [5]:
MATCH (tom:Person {name: "Tom Hanks"})-[in:ACTED_IN]->(tomHanksMovies) 
RETURN tom,in,tomHanksMovies

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| tom                                                   | in                                                                                                                | tomHanksMovies                                                                                                                                                                              |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [6]:
MATCH (cloudAtlas {title: "Cloud Atlas"})<-[:DIRECTED]-(directors)
RETURN directors.name

+-------------------+
| directors.name    |
+-------------------+
| "Lilly Wachowski" |
| "Lana Wachowski"  |
| "Tom Tykwer"      |
+-------------------+

3 rows available after 2 ms, consumed after another 0 ms

In [7]:
MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors)
RETURN coActors.name

+--------------------------+
| coActors.name            |
+--------------------------+
| "Kevin Bacon"            |
| "Bill Paxton"            |
| "Ed Harris"              |
| "Gary Sinise"            |
| "Helen Hunt"             |
| "Julia Roberts"          |
| "Philip Seymour Hoffman" |
| "Rosie O'Donnell"        |
| "Lori Petty"             |
| "Geena Davis"            |
| "Bill Paxton"            |
| "Madonna"                |
| "Nathan Lane"            |
| "Meg Ryan"               |
| "Rosie O'Donnell"        |
| "Victor Garber"          |
| "Bill Pullman"           |
| "Rita Wilson"            |
| "Meg Ryan"               |
| "Steve Zahn"             |
| "Dave Chappelle"         |
| "Parker Posey"           |
| "Greg Kinnear"           |
| "Meg Ryan"               |
| "Liv Tyler"              |
| "Charlize Theron"        |
| "Audrey Tautou"          |
| "Paul Bettany"           |
| "Ian McKellen"           |
| "Jim Broadbent"          |
| "Hugo Weaving"           |
| "Halle Berry

In [8]:
MATCH (people:Person)-[relatedTo]-(:Movie {title: "Cloud Atlas"}) 
RETURN people.name, Type(relatedTo), relatedTo

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| people.name        | Type(relatedTo) | relatedTo                                                                                                                                          |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| "Jessica Thompson" | "REVIEWED"      | [:REVIEWED {summary: "An amazing journey", rating: 95, _id_: 1838}[1370>1306]]                                                                     |
| "David Mitchell"   | "WROTE"         | [:WROTE {_id_: 1738}[1310>1306]]                                                                                                                   |
| "Stefan Arndt"     | "PRODUCED"      | [:PRODUCE

## Solve

You've heard of the classic "Six Degrees of Kevin Bacon"? That is simply a shortest path query called the "Bacon Path".

### Variable length patterns

In [9]:
MATCH (bacon:Person {name:"Kevin Bacon"})-[*1..4]-(hollywood)
RETURN DISTINCT hollywood

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| hollywood                                                                                                                                                                                                                                                |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| (:Person {name: "Helen Hunt", _id_: 1254, born: 1963})                                                                                                                                                                                         

### Built-in `shortestPath()` algorithm

In [10]:
MATCH p=shortestPath(
  (bacon:Person {name:"Kevin Bacon"})-[*]-(meg:Person {name:"Meg Ryan"})
)
RETURN p

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| p                                                                                                                                                                                                                                                                                                                                      

## Recommend

Let's recommend new co-actors for Tom Hanks. A basic recommendation approach is to find connections past an immediate neighborhood which are themselves well connected.

For Tom Hanks, that means:

  1. Find actors that Tom Hanks hasn't yet worked with, but his co-actors have.
  2. Find someone who can introduce Tom to his potential co-actor.
  
  
Extend Tom Hanks co-actors, to find co-co-actors who haven't work with Tom Hanks...

In [12]:
MATCH (tom:Person {name:"Tom Hanks"})-[:ACTED_IN]->(m)<-[:ACTED_IN]-(coActors),
      (coActors)-[:ACTED_IN]->(m2)<-[:ACTED_IN]-(cocoActors)
WHERE NOT (tom)-[:ACTED_IN]->()<-[:ACTED_IN]-(cocoActors) AND tom <> cocoActors
RETURN cocoActors.name AS Recommended, count(*) AS Strength ORDER BY Strength DESC

+---------------------------------+
| Recommended          | Strength |
+---------------------------------+
| "Tom Cruise"         | 5        |
| "Zach Grenier"       | 5        |
| "Cuba Gooding Jr."   | 4        |
| "Keanu Reeves"       | 4        |
| "Tom Skerritt"       | 3        |
| "Carrie-Anne Moss"   | 3        |
| "Val Kilmer"         | 3        |
| "Bruno Kirby"        | 3        |
| "Billy Crystal"      | 3        |
| "Carrie Fisher"      | 3        |
| "Kelly McGillis"     | 3        |
| "Anthony Edwards"    | 3        |
| "Laurence Fishburne" | 3        |
| "Jack Nicholson"     | 3        |
| "Michael Sheen"      | 2        |
| "Frank Langella"     | 2        |
| "Oliver Platt"       | 2        |
| "Emil Eifrem"        | 1        |
| "Jay Mohr"           | 1        |
| "Stephen Rea"        | 1        |
| "Ben Miles"          | 1        |
| "Renee Zellweger"    | 1        |
| "Regina King"        | 1        |
| "Kevin Pollak"       | 1        |
| "Rick Yune"          | 1  

Find someone to introduce Tom Hanks to Tom Cruise

In [13]:
MATCH (tom:Person {name:"Tom Hanks"})-[a:ACTED_IN]->(m)<-[b:ACTED_IN]-(coActors),
      (coActors)-[c:ACTED_IN]->(m2)<-[d:ACTED_IN]-(cruise:Person {name:"Tom Cruise"})
RETURN tom, a, m, b, coActors, c, m2, d, cruise

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| tom                                                   | a                            

## Clean up

When you're done experimenting, you can remove the movie data set.

Note:

  1. Nodes can't be deleted if relationships exist
  2. Delete both nodes and relationships together
 
**WARNING:** The following query will remove all nodes and relations in your database!

In [18]:
MATCH (n) DETACH DELETE n

0 rows available after 3 ms, consumed after another 0 ms
Deleted 171 nodes, Deleted 253 relationships

Check that the Movie Graph is gone...

In [19]:
MATCH (n) RETURN n

0 rows available after 7 ms, consumed after another 0 ms